# Deciphering Code with Character-Level RNN





## Dataset

In [1]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.preprocessing.sequence import pad_sequences

In [2]:
df = pd.read_csv('./data/Caesar.csv')
df

,Passwords,ciphertext,key
0,2428031609,3539142710,19
1,4531040045,0197606601,4
2,almigrana1,sdeayjsfs3,8
3,quiero95,nrfbol62,3
4,doitnow2,vgalfgo4,8
...,...,...,...
199995,dime168,otxp613,15
199996,!l0v3k3v!n,x6h9w9hz,14
199997,9870tmf05,3214ngz49,6
199998,CAMILA,IGSORG,20


In [3]:
df = df.dropna()[:120000]

In [4]:
df['Passwords'].apply(str)
df['ciphertext'].apply(str)
df

,Passwords,ciphertext,key
0,2428031609,3539142710,19
1,4531040045,0197606601,4
2,almigrana1,sdeayjsfs3,8
3,quiero95,nrfbol62,3
4,doitnow2,vgalfgo4,8
...,...,...,...
120011,ohshime.,vozoptl,19
120012,gabnice143,pjkwrln476,17
120013,brad81,wmvy36,5
120014,riky4ewer,evxl1rjre,13


## Preprocessing Data

In [5]:
def tokenize(x):
    x_tk = Tokenizer(char_level=True)
    x_tk.fit_on_texts(x)                 

    return x_tk.texts_to_sequences(x), x_tk

def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    
    return pad_sequences(x, maxlen=length, padding="post", truncating="post",)

### Preprocess Pipeline

In [6]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

In [7]:
preproc_code_sentences, preproc_plaintext_sentences, code_tokenizer, plaintext_tokenizer = preprocess(df['Passwords'], df['ciphertext'])

print('Data Preprocessed')

Data Preprocessed


In [8]:
preproc_code_sentences[0]

array([ 7, 17,  7, 15,  5, 13,  3, 19,  5, 12,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [9]:
len(code_tokenizer.word_index)+1

131

In [10]:
len(plaintext_tokenizer.word_index)+1

52

In [11]:
plaintext_tokenizer.word_index

{'1': 1,
 '0': 2,
 '2': 3,
 '3': 4,
 '9': 5,
 '8': 6,
 '4': 7,
 '5': 8,
 '7': 9,
 '6': 10,
 'q': 11,
 'x': 12,
 'j': 13,
 'z': 14,
 'f': 15,
 'w': 16,
 'v': 17,
 'c': 18,
 'g': 19,
 'd': 20,
 'u': 21,
 'k': 22,
 'p': 23,
 'b': 24,
 'h': 25,
 'm': 26,
 'y': 27,
 't': 28,
 's': 29,
 'n': 30,
 'r': 31,
 'o': 32,
 'l': 33,
 'i': 34,
 'e': 35,
 'a': 36,
 'à': 37,
 'ã': 38,
 '¹': 39,
 '³': 40,
 '²': 41,
 'â': 42,
 '¼': 43,
 'ä': 44,
 'ª': 45,
 'î': 46,
 'ï': 47,
 'µ': 48,
 'å': 49,
 'ö': 50,
 'á': 51}

In [12]:
preproc_code_sentences.shape

(120000, 255)

In [13]:
preproc_plaintext_sentences.shape

(120000, 244, 1)

# LSTM

In [14]:
from keras.layers import GRU, Input, Dense, TimeDistributed, RNN, LSTM
from keras.models import Model, Sequential
from keras.layers import Activation
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
import tensorflow


def simple_model(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    x = Input(shape=input_shape[1:])   
    seq = LSTM(units= 128, return_sequences = True, activation="tanh", name='Layer1')(x)
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer2'))(seq)
    model = Model(inputs = x, outputs = output)
    model.compile(optimizer='adam', loss= sparse_categorical_crossentropy, metrics=['accuracy'])
    model.summary()
    return model

tmp_x = pad(preproc_code_sentences, preproc_plaintext_sentences.shape[1]) 
tmp_x = tmp_x.reshape((-1, preproc_plaintext_sentences.shape[-2], 1))     

In [15]:
tmp_x.shape

(120000, 244, 1)

In [16]:
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 244, 1)]          0         
_________________________________________________________________
Layer1 (LSTM)                (None, 244, 128)          66560     
_________________________________________________________________
time_distributed (TimeDistri (None, 244, 52)           6708      
Total params: 73,268
Trainable params: 73,268
Non-trainable params: 0
_________________________________________________________________


In [17]:
simple_rnn_model.get_layer(name="Layer1").output.shape

TensorShape([None, 244, 128])

In [18]:
simple_rnn_model.fit(tmp_x, preproc_plaintext_sentences, batch_size=64, epochs=15, validation_split=0.3, use_multiprocessing=True)

Epoch 1/15
1313/1313 [==============================] - 49s 25ms/step - loss: 0.3123 - accuracy: 0.9634 - val_loss: 0.1233 - val_accuracy: 0.9663
Epoch 2/15
1313/1313 [==============================] - 31s 24ms/step - loss: 0.1219 - accuracy: 0.9665 - val_loss: 0.1199 - val_accuracy: 0.9667
Epoch 3/15
1313/1313 [==============================] - 33s 25ms/step - loss: 0.1192 - accuracy: 0.9668 - val_loss: 0.1176 - val_accuracy: 0.9670
Epoch 4/15
1313/1313 [==============================] - 31s 24ms/step - loss: 0.1170 - accuracy: 0.9671 - val_loss: 0.1161 - val_accuracy: 0.9671
Epoch 5/15
1313/1313 [==============================] - 31s 23ms/step - loss: 0.1159 - accuracy: 0.9672 - val_loss: 0.1151 - val_accuracy: 0.9673
Epoch 6/15
1313/1313 [==============================] - 31s 23ms/step - loss: 0.1144 - accuracy: 0.9675 - val_loss: 0.1143 - val_accuracy: 0.9673
Epoch 7/15
1313/1313 [==============================] - 31s 24ms/step - loss: 0.1138 - accuracy: 0.9675 - val_loss: 0.1135 -

In [19]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], plaintext_tokenizer))

0 8 2 8 0 3 1 7 0 9 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <P

In [20]:
df['Passwords'][0]

'2428031609'

# GRU

In [21]:
def simple_model1(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    x = Input(shape=input_shape[1:])   
    seq = GRU(units= 128, return_sequences = True, activation="tanh", name='Layer1')(x)  # output must be batchsize x timesteps x units
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer2'))(seq)
    model = Model(inputs = x, outputs = output)
    model.compile(optimizer='adam', loss= sparse_categorical_crossentropy, metrics=['accuracy'])
    model.summary()
    return model

In [22]:
simple_rnn_model1 = simple_model1(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 244, 1)]          0         
_________________________________________________________________
Layer1 (GRU)                 (None, 244, 128)          50304     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 244, 52)           6708      
Total params: 57,012
Trainable params: 57,012
Non-trainable params: 0
_________________________________________________________________


In [23]:
simple_rnn_model1.fit(tmp_x, preproc_plaintext_sentences, batch_size=64, epochs=15, validation_split=0.3, use_multiprocessing=True)

Epoch 1/15
1313/1313 [==============================] - 33s 23ms/step - loss: 0.3751 - accuracy: 0.9644 - val_loss: 0.1221 - val_accuracy: 0.9663
Epoch 2/15
1313/1313 [==============================] - 30s 23ms/step - loss: 0.1205 - accuracy: 0.9665 - val_loss: 0.1174 - val_accuracy: 0.9668
Epoch 3/15
1313/1313 [==============================] - 30s 23ms/step - loss: 0.1165 - accuracy: 0.9671 - val_loss: 0.1147 - val_accuracy: 0.9673
Epoch 4/15
1313/1313 [==============================] - 31s 24ms/step - loss: 0.1142 - accuracy: 0.9673 - val_loss: 0.1131 - val_accuracy: 0.9673
Epoch 5/15
1313/1313 [==============================] - 30s 23ms/step - loss: 0.1132 - accuracy: 0.9674 - val_loss: 0.1123 - val_accuracy: 0.9674
Epoch 6/15
1313/1313 [==============================] - 30s 23ms/step - loss: 0.1120 - accuracy: 0.9676 - val_loss: 0.1118 - val_accuracy: 0.9674
Epoch 7/15
1313/1313 [==============================] - 31s 23ms/step - loss: 0.1119 - accuracy: 0.9676 - val_loss: 0.1114 -

In [24]:
print(logits_to_text(simple_rnn_model1.predict(tmp_x[:1])[0], plaintext_tokenizer))

0 8 2 8 0 3 1 6 0 9 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <P

In [25]:
df['Passwords'][0]

'2428031609'

# SimpleRNN

In [26]:
from keras.layers import SimpleRNN 
def simple_model2(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    x = Input(shape=input_shape[1:])      
    seq = SimpleRNN(units= 128, return_sequences = True, activation="tanh", name='Layer1')(x)   
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer2'))(seq)    
    model = Model(inputs = x, outputs = output)    
    model.compile(optimizer='adam', loss= sparse_categorical_crossentropy, metrics=['accuracy'])    
    model.summary()    
    return model

In [27]:
simple_rnn_model2 = simple_model2(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 244, 1)]          0         
_________________________________________________________________
Layer1 (SimpleRNN)           (None, 244, 128)          16640     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 244, 52)           6708      
Total params: 23,348
Trainable params: 23,348
Non-trainable params: 0
_________________________________________________________________


In [28]:
simple_rnn_model2.fit(tmp_x, preproc_plaintext_sentences, batch_size=64, epochs=15, validation_split=0.3, use_multiprocessing=True)

Epoch 1/15
1313/1313 [==============================] - 182s 135ms/step - loss: 0.2353 - accuracy: 0.9461 - val_loss: 0.1235 - val_accuracy: 0.9662
Epoch 2/15
1313/1313 [==============================] - 177s 135ms/step - loss: 0.1224 - accuracy: 0.9663 - val_loss: 0.1201 - val_accuracy: 0.9666
Epoch 3/15
1313/1313 [==============================] - 177s 135ms/step - loss: 0.1192 - accuracy: 0.9668 - val_loss: 0.1183 - val_accuracy: 0.9670
Epoch 4/15
1313/1313 [==============================] - 181s 138ms/step - loss: 0.1171 - accuracy: 0.9672 - val_loss: 0.1161 - val_accuracy: 0.9671
Epoch 5/15
1313/1313 [==============================] - 177s 135ms/step - loss: 0.1158 - accuracy: 0.9674 - val_loss: 0.1150 - val_accuracy: 0.9673
Epoch 6/15
1313/1313 [==============================] - 178s 135ms/step - loss: 0.1146 - accuracy: 0.9675 - val_loss: 0.1141 - val_accuracy: 0.9674
Epoch 7/15
1313/1313 [==============================] - 177s 135ms/step - loss: 0.1138 - accuracy: 0.9676 - val_

In [29]:
print(logits_to_text(simple_rnn_model2.predict(tmp_x[:1])[0], plaintext_tokenizer))

0 5 2 3 0 3 1 6 0 3 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <P

In [30]:
df['Passwords'][0]

'2428031609'